# Imports

In [1]:
import pynq
import time
from pynq import PL
import xrfclk
import xrfdc
import numpy as np
# from fpbinary import FpBinary
import matplotlib.pyplot as plt
from scipy.fftpack import fft, fftfreq, fftshift
import mkidgen3 as g3
import mkidgen3.mkidpynq as mkidpynq
import mkidgen3.daccomb as daccomb
import mkidgen3
from mkidgen3.daccomb import generate_dac_comb
from mkidgen3.drivers import axiswitch, bintores, capture, ddc, dactable, axififo, rfdc
import scipy.io as sio

# Load the Overlay and Setup RFDC and MIG

In [2]:
#ol = pynq.Overlay('../Full_Channelizer/rst_jenny/full_chan_cap_nec_rst_two_intrcnt.bit', ignore_version=True)
ol = pynq.Overlay('RFDC_Cap_Full_Rate.bit', ignore_version=True)

#ol.download()
#print(f"PL Bitfile: {PL.bitfile_name}\nPL Timestamp: {PL.timestamp}\n"
#      f"Overlay timestamp: {ol.timestamp}  Loaded: {ol.is_loaded()}")

In [3]:
dacreplay = ol.dac_table_axim_0

# Setup RFDC and MIG

In [4]:
g3._gen3_overlay=ol
#xrfclk.set_ref_clks()
#ol.rfdc.start_clocks()
#ol.rfdc.rfdc_status(tell=True)
#print(f'Active ADC pair: {ol.rfdc.active_adc}')

## Setup DAC REPLAY

In [5]:
tones = np.array([0.8e9])
amplitudes = np.array([1.0])

In [6]:
dactable = generate_dac_comb(frequencies=tones, n_samples=2**19, sample_rate=4.096e9, amplitudes=amplitudes)
#{'iq': iq, 'frequencies': quantized_freqs, 'phases': phases}
qtones = dactable['frequencies']
qphases = dactable['phases']

In [7]:
dactable['iq'].real = 0

In [8]:
dacreplay.stop()

In [9]:
dacreplay.replay(dactable['iq'], fpgen=lambda x: (x*2**15).astype(np.uint16))

In [10]:
#dacreplay.quiet()

### Take ADC Snap

In [11]:
cap_result=ol.capture.capture_adc(2**19)

In [12]:
adc_capture_data = cap_result[:,0]/2**15 + 1j*cap_result[:,1]/2**15

In [13]:
cap_result.freebuffer()

In [14]:
sio.savemat('0_1_.mat', mdict={'adc_imag_800MHz':adc_capture_data})